In [228]:
import nltk
import numpy as np
import random
from nltk.corpus import wordnet as wn
from nltk.corpus import semcor

In [5]:
chunks = semcor.tagged_chunks(tag='both')
chunks
len(chunks)

778587

In [242]:
test_num = int(778587/5)
train_num = 778587 - test_num
shuffle = np.random.choice(778587, 778587, replace=False)
train_i = shuffle[:train_num]
test_i = shuffle[-1*test_num:]

In [290]:
print("training: ",len(train_i))
print("testing: ", len(test_i))

training:  622870
testing:  155717


In [261]:
## create dictionary for a segment of chunks
def count_synset(a,b,index):
    list_s = {}
    for i in range(a,b):
        chunk= chunks[index[i]]
        if chunk.label() is not None:
            if not isinstance(chunk.label(),str):
                ck_categ = chunk.label().synset().name().split(".")[0]
                ck_name = chunk.label().synset().name()
                if ck_categ in list_s:
                    if ck_name in list_s[ck_categ]:
                        list_s[ck_categ][ck_name] += 1
                    else:
                        list_s[ck_categ][ck_name] = 1
                else:
                    list_s[ck_categ] = {}
                    list_s[ck_categ][ck_name]=1
    return list_s


## merge two nested dictionary
def merge(d1, d2):
    if d1 =={}:
        return d2
    for k in d2:
        if k in d1 and isinstance(d1[k], dict) and isinstance(d2[k], dict):
            merge(d1[k], d2[k])
        else:
            d1[k] = d2[k] 
    return d1

## Convert Train data into dictionary 
- Count frequency of each sense

In [279]:
## count senses of first 600,000 chunks, as training data
import time
result = {}
for i in range(62):
    start = time.time()
    print(i)
    segment = count_synset(i*10000,(i+1)*10000,train_i)
    result = merge(result,segment)
    end = time.time()
    print(end-start)
segment = count_synset(620000,622869,train_i)
result = merge(result,segment)

0
63.23213744163513
1
60.80057191848755
2
62.392069578170776
3
68.61357855796814
4
70.12432432174683
5
57.63247537612915
6
58.62727117538452
7
57.583555459976196
8
58.0486044883728
9
61.036747217178345
10
56.623414516448975
11
61.312039375305176
12
57.08102893829346
13
56.36939597129822
14
58.090678691864014
15
65.90428614616394
16
65.13918542861938
17
63.127320289611816
18
59.81525778770447
19
58.44715118408203
20
59.30609202384949
21
59.90309810638428
22
58.9808611869812
23
58.640222787857056
24
58.89990305900574
25
60.465932846069336
26
65.40757966041565
27
58.9290292263031
28
58.12343978881836
29
58.579862117767334
30
60.12271451950073
31
58.956294298172
32
62.5788631439209
33
70.02761888504028
34
67.63263845443726
35
61.65708947181702
36
59.63716173171997
37
59.11623454093933
38
59.05408787727356
39
60.527588844299316
40
61.336925983428955
41
61.808279275894165
42
61.32623028755188
43
62.14174032211304
44
71.83173179626465
45
61.046517848968506
46
60.77556276321411
47
64.716930866

## Create prediction match for each word and sense

In [282]:
## select the sense with max number
reduced={}
for (k, v) in result.items():
    reduced[k] = max(result[k], key=result[k].get)

## Convert Test Data into dictionary

In [289]:
## count sense for test
test = {}
test = count_synset(0,155716,test_i)
len(test)

8765

## calculate accuracy for test data

In [287]:
## calculate accuracy for test data
tr, fl, na = 0, 0, 0
for k1, v1 in test.items():
    for k2, v2 in v1.items():
        if k1 in reduced:
            if k2 == reduced[k1]:
                tr += v2
            else:
                fl += v2
        else:
            na += v2

In [288]:
print(tr,fl,na)
print(tr/(tr+fl))

29494 14003 1504
0.6780697519369152


## calculate accuracy for train data

In [283]:
## calculate accuracy for train data
tr, fl, na = 0, 0, 0
for k1, v1 in result.items():
    for k2, v2 in v1.items():
        if k1 in reduced:
            if k2 == reduced[k1]:
                tr += v2
            else:
                fl += v2
        else:
            na += v2

In [284]:
print(tr,fl,na)
print(tr/(tr+fl))

17359 7484 0
0.6987481383085778
